<a href="https://colab.research.google.com/github/ronniesong0809/Digit-Classifier-App/blob/master/mnist_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

print("TensorFlow version:", tf.__version__)

mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

datagen = keras.preprocessing.image.ImageDataGenerator(
  rotation_range=30,
  width_shift_range=0.25,
  height_shift_range=0.25,
  shear_range=0.25,
  zoom_range=0.2
)

train_generator = datagen.flow(train_images, train_labels)
test_generator = datagen.flow(test_images, test_labels)

model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Dropout(0.25),
  keras.layers.Flatten(),
  keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_generator, epochs=5, validation_data=test_generator)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('mnist_v2.tflite', "wb") as f:
  f.write(tflite_model)

Step 1. Upload the private key (json file) for your service account and Initialize Firebase Admin

In [ ]:
import os
from google.colab import files
import firebase_admin
from firebase_admin import ml

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/content/' + fn
  projectID = fn.rsplit("-firebase")[0]
  firebase_admin.initialize_app(
      options={'projectId': projectID, 
               'storageBucket': projectID + '.appspot.com' })

Step 2.  Upload the model file to Cloud Storage

In [ ]:
source = ml.TFLiteGCSModelSource.from_keras_model(model, 'mnist_v2.tflite')
print (source.gcs_tflite_uri)

Step 3.  Deploy the model to Firebase

In [ ]:
model_format = ml.TFLiteFormat(model_source=source)

sdk_model_1 = ml.Model(display_name="mnist_v2", model_format=model_format)

firebase_model_1 = ml.create_model(sdk_model_1)
print(firebase_model_1.as_dict())

model_id = firebase_model_1.model_id
firebase_model_1 = ml.publish_model(model_id)